In [2]:
# Importing dataset from Digits
from sklearn.datasets import load_digits
# sklearn NearestNeighbors clustering and classification algorithms
from sklearn.neighbors import NearestNeighbors
from sklearn.neighbors import KNeighborsClassifier
# Square root function
from math import sqrt
# Plot library to display images, actually. Now is not used, but if you would uncomment some...
import matplotlib.pyplot as plt
# Numpy, copy. Don't remember why imported time, but let it be :-\
import numpy as np
import time
import copy

In [3]:
# Now loading digits
digits = load_digits()
# Separately loading digits with labels
digits_labels = load_digits(return_X_y=True)
digits_labels = digits_labels[1]

In [4]:
# You've asked for shape of dataset
print(digits.data.shape)
# It's 64 in a row for every image, so could expect

(1797, 64)


In [5]:
print(digits.images.shape)
# 8*8 for width and height

(1797, 8, 8)


In [6]:
# Data method gives us data in a row, and I doesn't want to care about feeding with 2-dimensional data
print(digits.data[0].shape)

(64,)


In [7]:
# Thought you want unsupervised stuff from us.
# In some case it worked. Look through for proofs.
# Applying KNN from Sklearn
# Fitting
nbrs = NearestNeighbors(n_neighbors=2, algorithm='auto').fit(digits.data)

In [8]:
# Getting distances and indeces
distances, indeces = nbrs.kneighbors(digits.data)

In [9]:
# Getting sparse graph, then reducing it to numbers of sparse cells
sparse_graph = nbrs.kneighbors_graph(digits.data).toarray()
sparse_nums = []
for entry in sparse_graph:
    nearest = []
    for (i,pos) in enumerate(entry):
        if pos == 1:
            nearest.append(i)
    sparse_nums.append(nearest)

In [10]:
# Actually, that's how it worked:
# I'm checking how many times algorithm marked different class objects as nearest.
fails = 0
for entry in sparse_nums:
    if digits_labels[entry[0]]!=digits_labels[entry[1]]:
        fails += 1

print('Number of wrong matches:', fails)
print('Percentage of wrong matches: <', int(100*fails/len(sparse_nums)),'%')

# Actually, the best result of that lab for me.

Number of wrong matches: 21
Percentage of wrong matches: < 1 %


In [11]:
# Split data into train classes(0-9):
# For this - creating the list
classes = []

# And the lists in it
for i in range(10):
    classes.append([])

# Just checked if int(float) would work properly
print(int(len(digits.images)*0.8))

# Then for every image of specific digit - put it in specific class for 80% of images.
for i in range(int(len(digits.images)*0.8)):
    classes[digits_labels[i]].append(digits.data[i])

1437


In [12]:
# Checking that classes are built correctly:
MAX, MIN = 0, 2000
for (i,clas) in enumerate(classes):
    print('Class',i,'size =',len(clas))
    MAX = max(len(clas),MAX)
    MIN = min(len(clas),MIN)
# Classes are already(and obviously) quite normalized

print('Deviation:', MAX-MIN)
print('Relative deviation: ~', int(100*(MAX-MIN)/((MAX+MIN)/2)),'%')

Class 0 size = 143
Class 1 size = 146
Class 2 size = 142
Class 3 size = 146
Class 4 size = 144
Class 5 size = 145
Class 6 size = 144
Class 7 size = 143
Class 8 size = 141
Class 9 size = 143
Deviation: 5
Relative deviation: ~ 3 %


In [13]:
# Now we could just fit it in binary style. Don't want to spent time on multiclass assosciations.
# Let's define euclidian distance for our case
def dist(a,b):
    summ = 0.0
    for i in range(len(a)):
        summ += (a[i]-b[i])**2
    return sqrt(summ)

In [14]:
# Checking if distance between classes is bigger than between entries of class
# The same time - let's create function for creating distances matrix:
def distances(data_recs):
    distances = []
    for (i,record) in enumerate(data_recs):
        distances.append([])
        for (j,other) in enumerate(data_recs):
            distances[i].append(dist(record,other))
    return distances

In [15]:
data = list(digits.data)

In [16]:
''' # Calculating distances. Takes forever to perform. Use next cell, if you don't want die old and unsatisfied.
# In next cell I just upload distances from file where I've recorded it for you.
# Thought also about json storing, but it needs to change IOPub data rate, which is... crap.
distances = distances(data)

# Creating file, would be real comma separated value file.
f = open('distances.csv', 'w')
for row in distances:
    for entry in row:
        f.write(str(entry)+',')
    f.write('\n')
f.close()'''

" # Calculating distances. Takes forever to perform. Use next cell, if you don't want die old and unsatisfied.\n# In next cell I just upload distances from file where I've recorded it for you.\n# Thought also about json storing, but it needs to change IOPub data rate, which is... crap.\ndistances = distances(data)\n\n# Creating file, would be real comma separated value file.\nf = open('distances.csv', 'w')\nfor row in distances:\n    for entry in row:\n        f.write(str(entry)+',')\n    f.write('\n')\nf.close()"

In [17]:
# Reading saved values for distances.
f = open('distances.csv', 'r')
distances = []
for line in f:
    distance = line.split(',')
    distance.pop(distance.index('\n'))
    for i, elem in enumerate(distance):
        distance[i] = float(elem)
    distances.append(distance)

In [18]:
# Now let's fit the sklearn kNN-classifier
X = digits.data
y = digits_labels
# Splitted for 1500 train and the rest to test
border = 1500
# There are even some poetry in comments
# Running for different number of neighbors:
for k in [1,2,3,4,5,6,7,8,9]:
    kNN = KNeighborsClassifier(n_neighbors=k)
    kNN.fit(X[:border], y[:border])

    # Then predict classes for the rest
    sk_kNN_labels = kNN.predict(X[border:])

    # Evaluating number of fails:
    # Haven't got yet how to apply confusion matrix to not-binary classifier.
    # Could be applied in correct_class/rest-like approach, but... it would be huge, so...
    fails = 0
    for i in range(len(y)-border):
        if sk_kNN_labels[i]!=y[border+i]:
            fails += 1
    print('For k = '+str(k)+' number of fails: '+str(fails))

For k = 1 number of fails: 16
For k = 2 number of fails: 13
For k = 3 number of fails: 12
For k = 4 number of fails: 12
For k = 5 number of fails: 13
For k = 6 number of fails: 16
For k = 7 number of fails: 16
For k = 8 number of fails: 17
For k = 9 number of fails: 17


In [230]:
# Could see optimal k was 3 or 4. This number could differ depending on data.
print('Also, accuracy with optimal k-number is: '+str(round(100 - (100*12/(len(y)-1500))))+'%')
print('Which is, actualy, very good.')
print('Best accuracy achieved in hand-written digits recognition is 98%, but...')
print('That accuracy achieved on quite different dataset.')

Also, accuracy with optimal k-number is: 96%
Which is, actualy, very good.
Best accuracy achieved in hand-written digits recognition is 98%, but...
That accuracy achieved on quite different dataset.


In [19]:
# Define the function, distances included as calculated to fasten process

def k_neighb(k, distances):
    k_neighbors = []
    distance = distances.copy()
    distance[distance.index(min(distance))]=np.inf
    for i in range(k):
        k_neighbors.append(distance.index(min(distance)))
        distance[distance.index(min(distance))]=np.inf
    return k_neighbors

def predict_label(neighbors, label):
    cnt = []
    for i in range(10):
        cnt.append(0)
    for neighb in neighbors:
        cnt[label[neighb]]+=1
    return(cnt.index(max(cnt)))

def classifyKNN(X, y, k, distances, border):
    pred = []
    for entry_num in range(len(y)-border):
        neighb = k_neighb(k, distances[entry_num+border])
        pred.append(predict_label(neighb, y))
    return pred

In [21]:
# Running my predictor:
border = 1500

for k in [1,2,3,4,5,6,7,8,9]:
    predicted_labels = classifyKNN(X, y, k, distances, border)

    fails = 0
    for i in range(len(y)-border):
        if predicted_labels[i]!=y[border+i]:
            fails += 1
    print('For k = '+str(k)+' number of fails: '+str(fails/(len(digits_labels)-1500)))

For k = 1 number of fails: 0.020202020202020204
For k = 2 number of fails: 0.020202020202020204
For k = 3 number of fails: 0.020202020202020204
For k = 4 number of fails: 0.020202020202020204
For k = 5 number of fails: 0.016835016835016835
For k = 6 number of fails: 0.020202020202020204
For k = 7 number of fails: 0.030303030303030304
For k = 8 number of fails: 0.030303030303030304
For k = 9 number of fails: 0.03367003367003367


In [226]:
# My predictor works better. Possibly cause I use floats for calculations, which takes a while, but provides me better accuracy.
# Cause sklearn kNN predictor was in "auto" mode which must provide best accuracy among different approaches to KNN.

In [221]:
# Obviously, k number influes my algorithm the same way. Best k couldn't be easily predicted, cause depends on
# Number and location of cross-class neighbors, that are closer to some element of other class, than it's real "classmates".

7

In [22]:
# Confusion table in consideration of: class "0"/not class "0"
total = len(y)-border

# Sklearn:
tp, tn, fp, fn = 0, 0, 0, 0
for i,label in enumerate(sk_kNN_labels):
    if (label == 0) and (y[border+i]==0):
        tp += 1
    if (label != 0) and (y[border+i]==0):
        fn += 1
    if (label == 0) and (y[border+i]!=0):
        fp += 1
    if (label != 0) and (y[border+i]!=0):
        tn += 1

print('Confusion table for sklearn algorithm(class0/notclass0):')
print('TP = '+ str(tp)+', TN = '+str(tn)+', FP = '+str(fp)+', FN = '+str(fn))
print('Accuracy = '+str((tp+tn)/float(total)))
print('Prevalence = '+str((tp+fn)/float(total)))
precision = (tp)/float(tp+fp)
print('Precision = '+str(precision))
print('FDR = '+str((fp)/float(tp+fp)))
print('FOR = '+str((fn)/float(fn+tn)))
print('NPV = '+str((tn)/float(fn+tn)))

TPR = (tp)/float(tp+fn)
print('TPR, Recall = '+str(TPR))
FPR = (fp)/float(fp+tn)
print('FPR = '+str(FPR))
LRP = TPR/float(FPR)
print('LR+ = '+str(LRP))

FNR = (fn)/float(tp+fn)
TNR = (tn)/float(fp+tn)
LRN = FNR/float(TNR)
print('FNR = '+str(FNR))
print('TNR = '+str(TNR))
print('LR- = '+str(LRN))

# print('DOR = '+str(LRP/LRN))
print("DOR - doesn't applicable as far as LR- is equal to zero")
print("F1 score = "+str(2/(1/TPR+1/precision)))

Confusion table for sklearn algorithm(class0/notclass0):
TP = 27, TN = 269, FP = 1, FN = 0
Accuracy = 0.9966329966329966
Prevalence = 0.09090909090909091
Precision = 0.9642857142857143
FDR = 0.03571428571428571
FOR = 0.0
NPV = 1.0
TPR, Recall = 1.0
FPR = 0.003703703703703704
LR+ = 270.0
FNR = 0.0
TNR = 0.9962962962962963
LR- = 0.0
DOR - doesn't applicable as far as LR- is equal to zero
F1 score = 0.9818181818181817


In [249]:
# MY:
tp, tn, fp, fn = 0, 0, 0, 0
for i,label in enumerate(predicted_labels):
    if (label == 0) and (y[border+i]==0):
        tp += 1
    if (label != 0) and (y[border+i]==0):
        fn += 1
    if (label == 0) and (y[border+i]!=0):
        fp += 1
    if (label != 0) and (y[border+i]!=0):
        tn += 1

print('Confusion table for my KNN algorithm(class0/notclass0):')
print('TP = '+ str(tp)+', TN = '+str(tn)+', FP = '+str(fp)+', FN = '+str(fn))
print('Accuracy = '+str((tp+tn)/float(total)))
print('Prevalence = '+str((tp+fn)/float(total)))
precision = (tp)/float(tp+fp)
print('Precision = '+str(precision))
print('FDR = '+str((fp)/float(tp+fp)))
print('FOR = '+str((fn)/float(fn+tn)))
print('NPV = '+str((tn)/float(fn+tn)))

TPR = (tp)/float(tp+fn)
print('TPR, Recall = '+str(TPR))
FPR = (fp)/float(fp+tn)
print('FPR = '+str(FPR))
#LRP = TPR/float(FPR)
print('LR+ - not applicable, cause FPR = 0')

FNR = (fn)/float(tp+fn)
TNR = (tn)/float(fp+tn)
LRN = FNR/float(TNR)
print('FNR = '+str(FNR))
print('TNR = '+str(TNR))
print('LR- = '+str(LRN))

# print('DOR = '+str(LRP/LRN))
print("DOR - doesn't applicable as far as LR- is equal to zero")
print("F1 score = "+str(2/(1/TPR+1/precision)))

Confusion table for my KNN algorithm(class0/notclass0):
TP = 27, TN = 270, FP = 0, FN = 0
Accuracy = 1.0
Prevalence = 0.09090909090909091
Precision = 1.0
FDR = 0.0
FOR = 0.0
NPV = 1.0
TPR, Recall = 1.0
FPR = 0.0
LR+ - not applicable, cause FPR = 0
FNR = 0.0
TNR = 1.0
LR- = 0.0
DOR - doesn't applicable as far as LR- is equal to zero
F1 score = 1.0
